In [1]:
from model_prediction import combine_roots_and_themes
import pandas as pd

# Read the CSV file into a DataFrame
input = pd.read_csv('sentences_author.csv')
input = input.sample(n=20, random_state=42)
print(input)

2024-12-12 00:16:47.175741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733962607.200150   45439 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733962607.206574   45439 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 00:16:47.227191: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/nana/anaconda3/envs/jitsupeer_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not foun

           author                                           sentence
0   Reviewer eS3u  This work proposes LSTNet, a self-supervised m...
5   Reviewer eS3u  The performance of aligned shape pairs under t...
33  Reviewer wiS9                                    See weaknesses.
13  Reviewer jP4i                  3) Experimental results are good.
19  Reviewer wiS9  This paper introduces LSTNet, which leverages ...
50  Reviewer Frem  2. The proposed method is straightforward and ...
36  Reviewer a6Ps  The SO(3)-invariant network design intrinsical...
26  Reviewer wiS9  2.2.2 Second, the so-called local shape transf...
44  Reviewer a6Ps  2. Will the network still be functional if the...
12  Reviewer jP4i  2) The idea of factorizing point cloud descrip...
54  Reviewer Frem  4. The whole method is mainly built upon the e...
3   Reviewer eS3u  The self- and cross-reconstruction training st...
34  Reviewer a6Ps  This paper attempts to register point cloud pr...
30  Reviewer wiS9  4. For the SO(3

In [2]:
results = combine_roots_and_themes(input)
results

I0000 00:00:1733962615.514895   45439 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5501 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:65:00.0, compute capability: 6.1
2024-12-12 00:16:56.757559: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at /home/nana/DASP_report_template/backend/models/attitude_root/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
2024-12-12 00:16:58.183619: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
Some layers from t

,Attitude_roots,Frequency,Descriptions,Comments
0,Substance(MET),0.6,Incomplete details on perfromance of the method,"[[Reviewer eS3u, [2. The proposed method is st..."
1,Clarity(MET),0.4,Unclear description of method,"[[Reviewer eS3u, [The reason why other methods..."
2,Substance(EXP),0.4,Experimental study not strong enough,"[[Reviewer eS3u, [3) Experimental results are ..."
3,Clarity(RWK),0.2,Improper writing of realted work section (as i...,"[[Reviewer eS3u, [4. For the SO(3)-equivariant..."
4,Meaningful-comparison(MET),0.2,Missing theoretical comparisons,"[[Reviewer eS3u, [4) Please compare the propos..."
5,Originality(MET),0.2,Limited novelty in theoretical contribution,"[[Reviewer eS3u, [4. The whole method is mainl..."
6,Meaningful-comparison(RWK),0.2,Missing baselines,"[[Reviewer eS3u, [4) Please compare the propos..."
7,Substance(DAT),0.2,Less datasets used,"[[Reviewer eS3u, [2. The proposed method is st..."
8,Originality(PDI),0.2,"The main reason is that from the narration, I ...","[[Reviewer eS3u, [2) The idea of factorizing p..."


In [7]:
results.to_csv('attitude_roots_themes.csv', index=False)

In [7]:
def create_clusters(row):
    # Extract root and themes
    root = row["attitude_root"]
    themes = row["attitude_themes"]
    
    # Combine root with each theme
    clusters = [f"{root}({theme})" for theme in themes]
    return clusters

df = results
df.loc[:, "clusters"] = df.apply(create_clusters, axis=1)
df = df.explode("clusters", ignore_index=True)

In [8]:
df

In [14]:
# Group by 'author' and 'clusters', and aggregate the sentences into a list
aggregated_df = df.groupby(['clusters', 'author'])['sentence'].apply(list).reset_index()
aggregated_df

In [16]:
final_df = aggregated_df.groupby('clusters').agg(
            comments=('sentence', lambda x: [[author, sentences] for author, sentences in zip(aggregated_df['author'], x)])
        ).reset_index()

In [17]:
final_df